In [90]:
import numpy as np
from scipy import sparse
import os
from utils import optm
from utils.functions import *
import numpy as np
from scipy import sparse
from importlib import reload



I = np.identity(2)
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


Sz = sparse.csr_matrix(Sz)
Sx = sparse.csr_matrix(Sx)
Sy = sparse.csr_matrix(Sy)
I = sparse.csr_matrix(I)

h1 = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real


h2 = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real


h3 = -(sparse.kron(sparse.kron(Sz,Sz,format='csr'), sparse.kron(I,I,format='csr'),format='csr') 
       + sparse.kron(sparse.kron(Sx,Sx,format='csr'), sparse.kron(I,I,format='csr'),format='csr')
       + sparse.kron(sparse.kron(Sy,Sy,format='csr'), sparse.kron(I,I,format='csr'),format='csr') 
     ).real

h4 = -(sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sz,Sz,format='csr'),format='csr') 
       + sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sx,Sx,format='csr'),format='csr')
       + sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sy,Sy,format='csr'),format='csr') 
     ).real




h1_ = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real

h2_ = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real



h = h1 + h2

h_ = h1_ + h2_

on_site = h3/4 + h4/4

In [91]:
J1 = 0.3
X1 = torch.tensor((J1 * h + on_site).toarray())
X1_prime = X1 - torch.eye(X1.shape[0])*(torch.diag(X1).min())

X2 = torch.tensor((J1 * h_ + on_site).toarray())
X2_prime = X2 - torch.eye(X2.shape[0])*(torch.diag(X2).min())

In [93]:
reload(optm)
model, gl = optm.optim_matrix_symm([X2_prime], 10000, optm_method = torch.optim.SGD, seed = 1, lr = 0.0001, momentum = 0.9,weight_decay = 0)

target loss : 0.650

---------- iteration start ----------
iteration :    0   loss : 0.924
iteration : 1000   loss : 0.823
iteration : 2000   loss : 0.768
iteration : 3000   loss : 0.753
iteration : 4000   loss : 0.750
iteration : 5000   loss : 0.750
iteration : 6000   loss : 0.750
iteration : 7000   loss : 0.750
iteration : 8000   loss : 0.750
iteration : 9000   loss : 0.750

 -------------- results --------------
target loss      : 0.650
loss before optm : 0.750
loss after optm  : 0.750


In [555]:
torch.det(torch.abs(model.matrix)).backward()

In [578]:
input1 = torch.abs(model.matrix)
input2 = torch.abs(model.matrix)
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)

In [580]:
model.matrix

tensor([[-0.3482-0.2558j,  0.0600+0.2633j,  0.0794+0.2549j, -0.0255+0.3171j,
          0.0600+0.2633j,  0.0793-0.1490j,  0.0667-0.1529j,  0.1417-0.1395j,
          0.0794+0.2549j,  0.0667-0.1529j,  0.0540-0.1558j,  0.1289-0.1483j,
         -0.0255+0.3171j,  0.1417-0.1395j,  0.1289-0.1483j,  0.2085-0.1067j],
        [ 0.0600+0.2633j, -0.3947-0.1984j,  0.0671+0.2867j,  0.0452+0.2743j,
          0.0793-0.1490j,  0.1055+0.2552j,  0.0855-0.1630j,  0.0907-0.1482j,
          0.0667-0.1529j,  0.1235+0.2434j,  0.0717-0.1672j,  0.0781-0.1530j,
          0.1417-0.1395j,  0.0284+0.3240j,  0.1536-0.1530j,  0.1543-0.1344j],
        [ 0.0794+0.2549j,  0.0671+0.2867j, -0.4420-0.1041j,  0.0923+0.2437j,
          0.0667-0.1529j,  0.0855-0.1630j,  0.1664+0.2299j,  0.0560-0.1530j,
          0.0540-0.1558j,  0.0717-0.1672j,  0.1816+0.2138j,  0.0435-0.1550j,
          0.1289-0.1483j,  0.1536-0.1530j,  0.1058+0.3171j,  0.1170-0.1521j],
        [-0.0255+0.3171j,  0.0452+0.2743j,  0.0923+0.2437j, -0.3470-0.255

In [546]:
X2_prime = X2_prime.type(torch.complex128)

In [527]:
# T = model.matrix
model._params = torch.nn.Parameter(model._params + 0.001 * model._params.grad)
# T - model.matrix

In [528]:
model._params

Parameter containing:
tensor([ 7.8543e-01,  7.8533e-01, -1.4895e-07,  1.0178e-04,  7.8543e-01,
         7.8533e-01], requires_grad=True)

In [533]:
E, V = torch.linalg.eigh(torch.abs(model.matrix @ X2_prime @ model.matrix.T))
E1, V1 = torch.linalg.eigh(torch.abs(X2_prime))

In [542]:
V[:,-1].conj()

tensor([-3.0149e-01, -2.4152e-05, -5.0257e-05, -8.1539e-08, -2.6583e-05,
        -4.5218e-01, -4.5229e-01, -2.6654e-05, -2.6586e-05, -4.5223e-01,
        -4.5234e-01, -2.6645e-05, -8.0982e-08, -5.0152e-05, -2.4125e-05,
        -3.0157e-01], dtype=torch.float64, grad_fn=<SelectBackward0>)

In [538]:
V1[:,-1]

tensor([ 0.0000e+00,  7.4138e-17,  1.9331e-16, -3.0151e-01, -4.1387e-17,
        -4.5227e-01, -4.5227e-01,  0.0000e+00, -4.1387e-17, -4.5227e-01,
        -4.5227e-01, -4.0580e-32, -3.0151e-01, -1.2174e-32, -6.0870e-32,
         0.0000e+00], dtype=torch.float64)

In [353]:
X1_prime = torch.rand(16,16, dtype = torch.float64)*2-1
X1_prime = X1_prime + X1_prime.T
X1_prime = X1_prime + torch.eye(16)*2

In [354]:
model2 = optm.unitary_solver([4,4],syms=True, seed = 101)
Y = torch.abs((model2.matrix @ X1_prime @ model2.matrix.T))
E, V = np.linalg.eigh(np.array(Y.detach()))
loss = torch.sum(torch.tensor(V[:,-1][:, None] @ V[:,-1][None, :]) * Y)
loss.backward()
# Y[1,1].backward()

In [355]:
model2._params.grad

tensor([-0.6779, -0.5087, -0.2851,  0.9110,  0.3692,  1.4537])